In [8]:
#### IMPORTAR BIBLIOTECAS NECESSÁRIAS
import csv
from typing import Union, Any
import tabula
import pandas as pd
import numpy as np
import re
import PyPDF2
from datetime import timedelta, time, datetime

import pendulum
from pendulum import now, yesterday, tomorrow, timezone, date
import dateutil
import os
import warnings
import requests
import pprint
import json
from tkinter import filedialog
from pandas import DataFrame
from pandas.io.parsers import TextFileReader

warnings.filterwarnings("ignore")

In [9]:

### LEITURA DO ARQUIVO DE SIGLAS SABRE
path = filedialog.askopenfilename()

print(path)

imported_data = open(path, 'rb')
pdf_reader = PyPDF2.PdfReader(imported_data)
num_paginas = len(pdf_reader.pages)
print(f"Número de páginas: {num_paginas}")

C:/Users/Ricardo/OneDrive/Área de Trabalho/SISTEMA AUDITORIA AERONAUTAS/AERONAUTAS/ARQUIVOS ORIGINAIS/MARCOS BELLINI/ESCALA IMPRESSÃO SITE.pdf
Número de páginas: 3


In [10]:
for i, pagina in enumerate(pdf_reader.pages):
    print(f"--- Página {i+1} ---")
    print(pagina.extract_text())
    
# SALVAR O TEXTO EXTRAÍDO EM UM ARQUIVO CSV
output_csv = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])
with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    for i, pagina in enumerate(pdf_reader.pages):
        texto = pagina.extract_text()
        linhas = texto.split('\n')
        
        for linha in linhas:
            csv_writer.writerow([linha])
    print(f"Texto extraído salvo em: {output_csv}")
    #print(f"A atividade é : {output_csv.split('/')[-1]}")  # Exibe apenas o nome do arquivo

# Fechar o arquivo PDF
imported_data.close()


--- Página 1 ---
MARCOS RODOLFO CELENZA BELLINI Escala Summary, 01 May - 01 Jun 2025
Generated 13 May 2025, 14:10 (VCP)
Wed, 30  April 2025 (Local time)th
Off (FR) 18:45 - 18:45 +1 VCP
Fri, 2 May 2025 (Local time)nd
Off (FR) 06:10 - 06:10 +1 VCP
Sat, 3 May 2025 (Local time)rd
Off (FR) 06:10 - 06:10 +1 VCP
Sun, 4 May 2025 (Local time)th
Apresentação 06:10 - 06:10 VCP
Reserva (SEA) 06:10 - 11:30 VCP
Release 11:30 - 11:30 VCP
Apresentação 13:00 - 13:00 VCP
Reserva (REX) 13:00 - 19:00 VCP
Release 19:00 - 19:00 VCP
Mon, 5 May 2025 (Local time)th
Apresentação 20:50 - 20:50 VCP
Reserva (R21) 20:50 - 23:50 VCP
Release 23:50 - 23:50 VCP
Tue, 6 May 2025 (Local time)th
Apresentação 20:50 - 20:50 VCP
Reserva (REA) 20:50 - 21:30 VCP
AD4484 (Extra a serviço) 21:30 - 22:42 VCP - CNF CA Flight Time: 01:12
Release 22:42 - 23:12 CNF
Hotel 23:42 - 20:15 +1 CNF
Wed, 7 May 2025 (Local time)th
Reserva (SBATZ) 14:00 - 20:45 CNF
Apresentação 20:45 - 21:34 CNF
AD4438 21:34 - 00:29 +1 CNF - BEL CA Flight Time: 

In [11]:
# Exibir mensagem de conclusão

print("Extração de texto concluída e salva em CSV.")

# Exibir o caminho do arquivo CSV salvo

print(f"Arquivo CSV salvo em: {output_csv}")

# Exibir o conteúdo do CSV
df = pd.read_csv(output_csv, header=None, encoding='utf-8')
print("Conteúdo do CSV:")
print(df[0:50])  # Exibe as primeiras linhas do DataFrame


Extração de texto concluída e salva em CSV.
Arquivo CSV salvo em: C:/Users/Ricardo/OneDrive/Área de Trabalho/SISTEMA AUDITORIA AERONAUTAS/AERONAUTAS/ARQUIVOS ORIGINAIS/MARCOS BELLINI/BELLINI MAIO 2025.csv
Conteúdo do CSV:
                                                    0
0   MARCOS RODOLFO CELENZA BELLINI Escala Summary,...
1                  Generated 13 May 2025, 14:10 (VCP)
2                  Wed, 30  April 2025 (Local time)th
3                       Off (FR) 18:45 - 18:45 +1 VCP
4                      Fri, 2 May 2025 (Local time)nd
5                       Off (FR) 06:10 - 06:10 +1 VCP
6                      Sat, 3 May 2025 (Local time)rd
7                       Off (FR) 06:10 - 06:10 +1 VCP
8                      Sun, 4 May 2025 (Local time)th
9                      Apresentação 06:10 - 06:10 VCP
10                    Reserva (SEA) 06:10 - 11:30 VCP
11                          Release 11:30 - 11:30 VCP
12                     Apresentação 13:00 - 13:00 VCP
13                    

In [12]:
# a estrutura do CSV deve conter as colunas:
df_escala_inicial = pd.DataFrame({
    'Activity': pd.Series(dtype='category'),
    'Checkin': pd.Series(dtype='datetime64[ns]'),
    'Start': pd.Series(dtype='datetime64[ns]'),
    'Dep': pd.Series(dtype='category'),
    'Arr': pd.Series(dtype='category'),
    'End': pd.Series(dtype='datetime64[ns]'),
    'Checkout': pd.Series(dtype='datetime64[ns]'),
    'CAT': pd.Series(dtype='category')
})

df_escala_inicial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Activity  0 non-null      category      
 1   Checkin   0 non-null      datetime64[ns]
 2   Start     0 non-null      datetime64[ns]
 3   Dep       0 non-null      category      
 4   Arr       0 non-null      category      
 5   End       0 non-null      datetime64[ns]
 6   Checkout  0 non-null      datetime64[ns]
 7   CAT       0 non-null      category      
dtypes: category(4), datetime64[ns](4)
memory usage: 564.0 bytes



# INICIAR A LEITURA DO ARQUIVO CSV E PROCESSAMENTO DOS DADOS
# LOOP NO ARQUIVO CSV PARA PROCESSAR AS LINHAS E ADICIONAR AO DataFrame
for index, row in df_escala_inicial.iterrows():
    linha = row[0]  # Acessa a coluna que contém o texto extraído
    if linha.strip():  # Verifica se a linha não está vazia

        # Dividir a linha em colunas usando regex para capturar os campos
        #colunas = re.split(r'\s{2,}', linha.strip())
        
        # SE A LINHA CONTIVER ','
        if ',' in linha:
            # colocar os valores apos a virgula até encontrar '(' na variavel var_data
            # Extrair os valores após a vírgula até encontrar o caractere '
            after_comma = linha.split(',', 1)[1]  # pega tudo após a primeira vírgula
            var_data = after_comma.split("(")[0].strip()
            
            print(f"VAR DATA: {var_data}")
            
            # imprimir 80 caracteres '-'
            print('-' * 80)
            
            # ir para a proxima linha
            continue
        # caso a linha possua Off
        if 'Off' in linha:
            # Extrair os valores após ( até encontrar o caractere )
            after_parenthesis = linha.split('(', 1)[1]  # pega tudo após o primeiro '('
            var_activity = after_parenthesis.split(")")[0].strip()  # pega tudo até o primeiro ')'
            
            print(f"VAR ACTIVITY: {var_activity}")
            
            # na proxima coluna temos o horário de checkin
            var_checkin = after_parenthesis.split(")")[1].strip()  # pega tudo após o primeiro ')'      
            
            # Extrair o horário de checkin e start
            var_checkin_time = var_checkin.split()[0]  # pega o primeiro horário
            var_start_time = var_checkin_time 
            
            print(f"VAR CHECKIN TIME: {var_checkin_time}")  
            print(f"VAR START TIME: {var_start_time}")  

            
            print('-' * 80)
            # ir para a proxima linha
            continue  
                        
        if 'Apresentacao' not in linha:
            # caso a proxima linha inicie com Reserva
            if index + 1 < len(df):  # Verifica se há uma próxima linha
                proxima_linha = df.iloc[index + 1, 0]  # Acessa a próxima linha
                if proxima_linha.startswith('Reserva'):
                    # Extrair a activity da próxima linha
                    proxima_colunas = proxima_linha.split()  # Divide a linha em colunas usando espaços como delimitador
                    var_activity = proxima_colunas[0]  # A primeira coluna é a atividade
                    var_start_time = proxima_colunas[1]  # A segunda coluna é o horário de início  
                    # Extrair o horário de checkin da próxima linha
                    var_checkin_time = proxima_colunas[1]  # A segunda coluna é o horário de checkin
                    # retroceder uma linha
                    #colunas = df.iloc[index - 1, 0].split()  # Divide a linha anterior em colunas usando espaços como delimitador
            
            
            ##### PAREI AQUI FALTA COMPLETAR O DIA 4 QUE TEM RESERVA INTERRUPÇÃO E OUTRA RESERVA 
                                
            var_checkin_time = var_checkin.split()[0]  # pega o primeiro horário
            # tudo da primeira coluna na linha atual
            colunas = linha.split()  # Divide a linha em colunas usando espaços como delimitador
            var_activity = colunas[0]  # A primeira coluna é a atividade 

            print(f"VAR ACTIVITY: {var_activity}")
            print(f"VAR CHECKIN TIME: {var_checkin_time}")                  
            print(f"VAR START TIME: {var_start_time}")            
            #print('-' * 80)
            # ir para a proxima linha
            continue  
            # caso a activity seja Reserva, pegar o horário de checkin
            
            # retroceder uma linha
            colunas = df.iloc[index - 1, 0].split()  # Divide a linha anterior em colunas usando espaços como delimitador
        """        
        # Verificar se a linha tem o número correto de colunas
        if len(colunas) >= 1: #7:
            # Criar um dicionário com os dados da linha
            dados = {
                'Activity': colunas[0],
                'Checkin': pd.to_datetime(colunas[1], errors='coerce'),
                'Start': pd.to_datetime(colunas[2], errors='coerce'),
                'Dep': colunas[3],
                'Arr': colunas[4],
                'End': pd.to_datetime(colunas[5], errors='coerce'),
                'Checkout': pd.to_datetime(colunas[6], errors='coerce'),
                'CAT': colunas[7] if len(colunas) > 7 else None
            }
            # Adicionar os dados ao DataFrame
            df_escala_inicial = df_escala_inicial.append(dados, ignore_index=True)
        """
        # gravando os dados no DataFrame
        df_escala_inicial = df_escala_inicial.append({
            'Activity': var_activity,
            'Checkin': pd.to_datetime(var_checkin_time, errors='coerce'),
            'Start': pd.to_datetime(var_start_time, errors='coerce'),
            'Dep': None,  # Coluna Dep não está sendo preenchida
            'Arr': None,  # Coluna Arr não está sendo preenchida
            'End': None,  # Coluna End não está sendo preenchida
            'Checkout': None,  # Coluna Checkout não está sendo preenchida
            'CAT': None  # Coluna CAT não está sendo preenchida
        }, ignore_index=True)
        
print(f"DATAFRAME CRIADO: {(df_escala_inicial)}")

print(df_escala_inicial.head(50))  # Exibe as primeiras 50 linhas do DataFrame resultante


m# Exibir o DataFrame resultante completo

print("DataFrame completo:")
print(df_escala_inicial.to_string(index=False))  # Exibe o DataFrame completo sem o índice

# Exibir o número de linhas e colunas do DataFrame resultante
print(f"Número de linhas: {df_escala_inicial.shape[0]}, Número de colunas: {df_escala_inicial.shape[1]}")


# Exibir informações do DataFrame resultante
print("Informações do DataFrame resultante:")
print(df_escala_inicial.info())  # Exibe informações sobre o DataFrame, como tipos de dados e número de entradas

# Exibir estatísticas descritivas do DataFrame resultante
print("Estatísticas descritivas do DataFrame resultante:")
print(df_escala_inicial.describe(include='all'))  # Exibe estatísticas descritivas, incluindo colunas não numéricas


# Exibir o DataFrame como uma tabela formatada

#Exibir o DataFrame completo
#print("DataFrame completo:")  
#print(df.to_string(index=False)

# Exibir o número de linhas e colunas do DataFrame
print(f"Número de linhas: {df_escala_inicial.shape[0]}, Número de colunas: {df_escala_inicial.shape[1]}") 

# Exibir informações do DataFrame
print("Informações do DataFrame:")
print(df.info())  # Exibe informações sobre o DataFrame, como tipos de dados e número de entradas


# Exibir estatísticas descritivas do DataFrame
print("Estatísticas descritivas do DataFrame:") 
print(df.describe(include='all'))  # Exibe estatísticas descritivas, incluindo colunas não numéricas


# Exibir o DataFrame como uma tabela formatada
print("DataFrame formatado:")
print(df.to_markdown(index=False))  # Exibe o DataFrame formatado como uma tabela Markdown


# Exibir o DataFrame como uma tabela HTML
print("DataFrame como tabela HTML:")Markdown
print(df.to_html(index=False))  # Exibe o DataFrame formatado como uma tabela HTML


# Exibir o DataFrame como uma tabela LaTeX  
print("DataFrame como tabela LaTeX:")
print(df.to_latex(index=False))  # Exibe o DataFrame formatado como uma tabela LaTeX

In [13]:
##### limpar o terminal
import os
os.system('cls' if os.name == 'nt' else 'clear')

0

# Exibir o DataFrame como uma tabela JSON
print("DataFrame como tabela JSON:")
print(df_escala_inicial.to_json(orient='records', lines=True))  # Exibe o DataFrame formatado como uma tabela JSON


# Exibir o DataFrame como uma tabela Excel
#print("DataFrame como tabela Excel:")
#excel_output = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx")])
#df.to_excel(excel_output, index=False)  # Salva o DataFrame como um arquivo Excel 
#print(f"DataFrame salvo como Excel em: {excel_output}")


# Exibir o DataFrame como uma tabela Parquet  
print("DataFrame como tabela Parquet:")
parquet_output = filedialog.asksaveasfilename(defaultextension=".parquet", filetypes=[("Parquet files", "*.parquet")])  
#df.to_parquet(parquet_output, index=False)  # Salva o DataFrame como um arquivo Parquet
print(f"DataFrame salvo como Parquet em: {parquet_output}") 


# Exibir o DataFrame como uma tabela Feather
print("DataFrame como tabela Feather:")
feather_output = filedialog.asksaveasfilename(defaultextension=".feather", filetypes=[("Feather files", "*.feather")])
df.to_feather(feather_output)  # Salva o DataFrame como um arquivo Feather  
print(f"DataFrame salvo como Feather em: {feather_output}")


# Exibir o DataFrame como uma tabela HDF5
print("DataFrame como tabela HDF5:")
hdf5_output = filedialog.asksaveasfilename(defaultextension=".h5", filetypes=[("HDF5 files", "*.h5")])
df.to_hdf(hdf5_output, key='data', mode='w')  # Salva o DataFrame como um arquivo HDF5  
print(f"DataFrame salvo como HDF5 em: {hdf5_output}")


# Exibir o DataFrame como uma tabela SQL
print("DataFrame como tabela SQL:")
import sqlite3
sql_output = filedialog.asksaveasfilename(defaultextension=".db", filetypes=[("SQLite files", "*.db")])
conn = sqlite3.connect(sql_output)  # Conecta ou cria um banco de dados SQLite
df.to_sql('tabela', conn, if_exists='replace', index=False)  # Salva o DataFrame como uma tabela SQL
print(f"DataFrame salvo como SQL em: {sql_output}") 


# Fechar a conexão com o banco de dados SQLite
conn.close()


# Exibir o DataFrame como uma tabela JSON Lines
print("DataFrame como tabela JSON Lines:")
print(df.to_json(orient='records', lines=True))  # Exibe o DataFrame formatado como uma tabela JSON Lines


# Exibir o DataFrame como uma tabela CSV
print("DataFrame como tabela CSV:") 
print(df.to_csv(index=False))  # Exibe o DataFrame formatado como uma tabela CSV


# Exibir o DataFrame como uma tabela TSV  
print("DataFrame como tabela TSV:")
print(df.to_csv(sep='\t', index=False))  # Exibe o DataFrame formatado como uma tabela TSV


# Exibir o DataFrame como uma tabela XML
print("DataFrame como tabela XML:")


# Carregar o DataFrame a partir do arquivo CSV
df_xml = pd.read_csv(output_csv, header=None, encoding='utf-8')
#print(df_xml.to_xml(index=False))  # Exibe o DataFrame formatado como uma tabela XML
# Exibir o DataFrame como uma tabela Markdown
print("DataFrame como tabela Markdown:")
print(df.to_markdown(index=False))  # Exibe o DataFrame formatado como uma tabela Markdown
display(df_xml)

In [14]:
# iniciar a leitura de cada linha do CSV e processar os dados
# encontrar a primeira linha que contém a palavra correspondente as 3 primeiras letras do dia da semana em ingles no inicio da linha

path = filedialog.askopenfilename(title="Selecione o arquivo CSV", filetypes=[("CSV files", "*.csv")])

print(f"Arquivo selecionado: {path}")

# Verificar se o caminho do arquivo foi selecionado
if not path:
    print("Nenhum arquivo selecionado.")
    exit()

df_escala_inicial = pd.read_csv(path, header=None)
print("DataFrame carregado com sucesso.")
#print(f"DataFrame inicial: {df_escala_inicial.head(10)}")  # Exibe as primeiras 10 linhas do DataFrame

# nome do arquivo CSV
var_nome_arquivo = linha = df_escala_inicial.iloc[0, 0]  # Extrai o nome do arquivo do caminho completo
print(f"Nome do arquivo: {var_nome_arquivo}")

# Loop para ler o arquivo CSV até encontrar uma linha que comece com um dia da semana
linha = df_escala_inicial.iloc[2, 0]  # Acessa a primeira linha do DataFrame
n = 2

while True:
    # verificar se chegou ao final do DataFrame
    if n >= len(df_escala_inicial):
        print("Chegou ao final do DataFrame, encerrando o loop.")
        break

    linha = df_escala_inicial.iloc[n, 0]
    
    if linha.startswith('Mon') or linha.startswith('Tue') or linha.startswith('Wed') or linha.startswith('Thu') or linha.startswith('Fri') or linha.startswith('Sat') or linha.startswith('Sun'):
        print(f"Linha encontrada: {linha}")
        # variavel que contem a data
        var_data = linha.split()[1] + "/" + linha.split()[2] + "/" + linha.split()[3] # Pega a data e hora da linha  
        print(f"Data encontrada HOJE: {var_data}")
        # avançar para a próxima linha do CSV
        n += 1
        linha = df_escala_inicial.iloc[n, 0]
        print(f"Próxima linha HOJE: {linha}")    
        
        # se a linha iniciar com a palavra 'Apresentação', continuar o loop
        if linha.startswith('Apresentação'):
            print("Linha iniciada com 'Apresentação', continuando o loop...")
            n += 1
            continue

        # se a linha iniciar com a palavra 'Off', parar o loop
        if linha.startswith('Off'):
            print("Linha iniciada com 'Off'")
            
            # variavel que contem a atividade sem os parênteses
            var_activity = linha.split()[1]
            # retirar os parênteses da atividade
            var_activity = var_activity.replace('(', '').replace(')', '')
            print(f"Atividade encontrada: {var_activity}")
            
            var_hora_checkin = linha.split()[2]  # Pega o horário de checkin da linha
            print(f"Horário de checkin encontrado: {var_hora_checkin}")
            
            var_hora_start = var_hora_checkin  # Pega o horário de início da linha
            print(f"Horário de início encontrado: {var_hora_start}")
            
            var_hora_end = linha.split()[2]  # Pega o horário de fim da linha
            print(f"Horário de fim encontrado: {var_hora_end}")
            
            var_hora_checkout = linha.split()[2]  # Pega o horário de checkout da linha
            print(f"Horário de checkout encontrado: {var_hora_end}")                  
        
            # montar as variaveis de data e hora
            var_data_hora_checkin = f"{var_data} {var_hora_checkin}"
            var_data_hora_start = f"{var_data} {var_hora_start}"
            # as variaveis de data e hora de fim e checkout deverão ser no dia seguinte
            var_data_hora_end = (datetime.strptime(var_data, "%d/%B/%Y") + timedelta(days=1)).strftime("%d/%B/%Y") + f" {var_hora_end}" 
            var_data_hora_checkout = (datetime.strptime(var_data, "%d/%B/%Y") + timedelta(days=1)).strftime("%d/%B/%Y") + f" {var_hora_checkout}"
            
            print(f"Data e hora de checkin: {var_data_hora_checkin}")
            print(f"Data e hora de início: {var_data_hora_start}")
            print(f"Data e hora de fim: {var_data_hora_end}")
            print(f"Data e hora de checkout: {var_data_hora_checkout}")
            
            # variaveis para Dep e Arr
            var_dep = linha.split()[6] #if len(linha.split()) > 3 else None
            var_arr = linha.split()[6] #if len(linha.split()) > 4 else None  
            print(f"Dep: {var_dep}")
            print(f"Arr: {var_arr}")    
            
            # imprimir 80 caracteres '-'
            print('-' * 80)
            
            # mostrar todas as variaveis numa mesma string
            print(f"Ativity: {var_activity}, Checkin: {var_data_hora_checkin}, Start: {var_data_hora_start}, Dep: {var_dep}, Arr: {var_arr}, End: {var_data_hora_end}, Checkout: {var_data_hora_checkout}")

        # abrir um arquivo CSV para salvar os dados
        # nome do arquivo CSV
        if not var_nome_arquivo.lower().endswith('.csv'):
            var_nome_arquivo = var_nome_arquivo + '.csv'
        with open(var_nome_arquivo, 'a', newline='', encoding='utf-8') as csvfile:
            csv_writer = csv.writer(csvfile)
            # escrever o cabeçalho se o arquivo estiver vazio
            if csvfile.tell() == 0:
                csv_writer.writerow(['Activity', 'Checkin', 'Start', 'Dep', 'Arr', 'End', 'Checkout'])
            # escrever os dados no arquivo CSV
            csv_writer.writerow([var_activity, var_data_hora_checkin, var_data_hora_start, var_dep, var_arr, var_data_hora_end, var_data_hora_checkout])

            n = n + 1
            # atualizar a linha para a próxima iteração
            # imprimir 80 caracteres '-'
            print('-' * 80)
    else:
        print(f"Linha não iniciada com um dia da semana: {linha}")
        n += 1
        
        var_activity = linha.split()[0]
        print(f"Atividade encontrada: {var_activity}")
        # verificar se a atividade é 'Apresentação' ou 'Release'
        
        
        
        #if var_activity.startswith('Apresentação') or var_activity.startswith('Release'):
        #    print(f"Atividade iniciada com 'Apresentação' ou 'Release', continuando o loop...")
        #    n += 1
        #    continue
        
        
        # se var_activity for 'Apresentação', 'Release' continuar o loop
        
        # verificar se a atividade inicia com "AD"
        #if var_activity.startswith('AD'):
            # pegar o horario de apresentação da linha anterior
            #n -= 1
            #linha_anterior = df_escala_inicial.iloc[n, 0]
            
            
            
            
        var_hora_checkin = linha.split()[4]
        var_hora_start = linha.split()[4]
        var_hora_end = linha.split()[4]
        var_hora_checkout = linha.split()[4]
        print(f"Horário de checkin encontrado: {var_checkin}")
        
        var_start = linha.split()[2]
        print(f"Horário de início encontrado: {var_start}")
        
        var_end = linha.split()[4]
        print(f"Horário de fim encontrado: {var_end}")
        
        var_checkout = linha.split()[4]
        print(f"Horário de checkout encontrado: {var_checkout}")
        
        # montar as variaveis de data e hora
        # se var_activity iniciar com 'AD'
        #if var_activity.startswith('AD'):
            # Apenas concatena a data e hora como string
        #    var_data_hora_end = f"{var_data} {var_end}"
        #    var_data_hora_checkout = f"{var_data} {var_checkout}"
        #else: 
        var_data_hora_checkin = f"{var_data} {var_checkin}"
        var_data_hora_start = f"{var_data} {var_start}"
        var_data_hora_end = f"{var_data} {var_end}"
        var_data_hora_checkout = f"{var_data} {var_checkout}"
        
        print(f"Data e hora de checkin: {var_data_hora_checkin}")
        print(f"Data e hora de início: {var_data_hora_start}")
        print(f"Data e hora de fim: {var_data_hora_end}")
        print(f"Data e hora de checkout: {var_data_hora_checkout}")
        
        # variaveis para Dep e Arr
        #if var_activity.startswith('AD'):
        #    var_dep = linha.split()[5] if len(linha.split()) > 5 else None
        #    var_arr = linha.split()[7] if len(linha.split()) > 6 else None
        #else:
        var_dep = linha.split()[2] #if len(linha.split()) > 5 else None
        var_arr = linha.split()[4] #if len(linha.split()) > 6 else None
        
        print(f"Dep: {var_dep}")
        print(f"Arr: {var_arr}")
        
        # imprimir 80 caracteres '-'
        print('-' * 80)
        
        # mostrar todas as variaveis numa mesma string
        print(f"Ativity: {var_activity}, Checkin: {var_data_hora_checkin}, Start: {var_data_hora_start}, Dep: {var_dep}, Arr: {var_arr}, End: {var_data_hora_end}, Checkout: {var_data_hora_checkout}")
          
        # abrir um arquivo CSV para salvar os dados
        # nome do arquivo CSV
        if not var_nome_arquivo.lower().endswith('.csv'):
            var_nome_arquivo = var_nome_arquivo + '.csv'
        with open(var_nome_arquivo, 'a', newline='', encoding='utf-8') as csvfile:
            csv_writer = csv.writer(csvfile)
            # escrever o cabeçalho se o arquivo estiver vazio
            if csvfile.tell() == 0:
                csv_writer.writerow(['Activity', 'Checkin', 'Start', 'Dep', 'Arr', 'End', 'Checkout'])
            # escrever os dados no arquivo CSV
            csv_writer.writerow([var_activity, var_data_hora_checkin, var_data_hora_start, var_dep, var_arr, var_data_hora_end, var_data_hora_checkout])
        print(f"Dados salvos no arquivo: {var_nome_arquivo}")


     
        
        # Exibir mensagem de conclusão

Arquivo selecionado: C:/Users/Ricardo/OneDrive/Área de Trabalho/SISTEMA AUDITORIA AERONAUTAS/AERONAUTAS/ARQUIVOS ORIGINAIS/MARCOS BELLINI/BELLINI MAIO 2025.csv
DataFrame carregado com sucesso.
Nome do arquivo: MARCOS RODOLFO CELENZA BELLINI Escala Summary, 01 May - 01 Jun 2025
Linha encontrada: Wed, 30  April 2025 (Local time)th
Data encontrada HOJE: 30/April/2025
Próxima linha HOJE: Off (FR) 18:45 - 18:45 +1 VCP
Linha iniciada com 'Off'
Atividade encontrada: FR
Horário de checkin encontrado: 18:45
Horário de início encontrado: 18:45
Horário de fim encontrado: 18:45
Horário de checkout encontrado: 18:45
Data e hora de checkin: 30/April/2025 18:45
Data e hora de início: 30/April/2025 18:45
Data e hora de fim: 01/May/2025 18:45
Data e hora de checkout: 01/May/2025 18:45
Dep: VCP
Arr: VCP
--------------------------------------------------------------------------------
Ativity: FR, Checkin: 30/April/2025 18:45, Start: 30/April/2025 18:45, Dep: VCP, Arr: VCP, End: 01/May/2025 18:45, Checkou

NameError: name 'var_checkin' is not defined